In [36]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install scipy
!pip install openpyxl

!pip install mathjax


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import optimize

#import scipy as sp

#plt.rcParams['text.usetex'] = True

### 5.1

In [37]:
Messwerte = pd.read_excel (r'V8 Brennstoffzelle.xlsx', sheet_name='5.1')
print (Messwerte)

          I      U
0   0.61400  3.510
1   0.55900  3.480
2   0.50500  3.443
3   0.40200  3.380
4   0.25800  3.288
5   0.18400  3.233
6   0.10800  3.164
7   0.98800  3.712
8   1.35900  3.880
9   1.50000  3.930
10  1.40300  3.880
11  1.20200  3.760
12  1.00400  3.710
13  1.01400  3.680
14  0.88900  3.620
15  0.73800  3.548
16  0.47800  3.405
17  0.38400  3.352
18  0.03970  3.138
19  0.03314  3.062
20  0.00920  2.988
21  1.08900  3.712
22  1.30100  3.859
23  0.80100  3.589
24  0.68200  3.525


In [38]:
IU = Messwerte.to_numpy()
IU[:, 0]= IU[:, 0] * 1000
IU

array([[ 614.   ,    3.51 ],
       [ 559.   ,    3.48 ],
       [ 505.   ,    3.443],
       [ 402.   ,    3.38 ],
       [ 258.   ,    3.288],
       [ 184.   ,    3.233],
       [ 108.   ,    3.164],
       [ 988.   ,    3.712],
       [1359.   ,    3.88 ],
       [1500.   ,    3.93 ],
       [1403.   ,    3.88 ],
       [1202.   ,    3.76 ],
       [1004.   ,    3.71 ],
       [1014.   ,    3.68 ],
       [ 889.   ,    3.62 ],
       [ 738.   ,    3.548],
       [ 478.   ,    3.405],
       [ 384.   ,    3.352],
       [  39.7  ,    3.138],
       [  33.14 ,    3.062],
       [   9.2  ,    2.988],
       [1089.   ,    3.712],
       [1301.   ,    3.859],
       [ 801.   ,    3.589],
       [ 682.   ,    3.525]])

In [39]:
plt.plot(IU[:, 1], IU[:, 0])


RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 432x288 with 1 Axes>

### 5.2

In [ ]:
Messwerte = pd.read_excel (r'V8 Brennstoffzelle.xlsx', sheet_name='5.2')
print (Messwerte)

IUtV0VE= Messwerte.to_numpy()

V_experiment = IUtV0VE[:, 4] - IUtV0VE[:, 3]

p = 760 / 1013.15 * 555555555
T = 666666  + 273.15

def func_V_theoretisch(I, t, T, p):
    return 88888 * I * t / (96485 * 2 * p) 
    
eta_Faraday = V_experiment / V_theoretisch

Heizwert = ( 11920 - 12770 ) / 20 * T

eta_Energie = Heizwert * V_experiment / (I * U * t)